In [ ]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_IT.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IT/functions_IT_v1.ipynb

# 1. Data Loading

In [ ]:
ds = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/ssh_igw_*.nc",combine='nested',concat_dim='time_counter',parallel=True)

In [ ]:
time_start = '2014-05-01 00:30'
#time_end = '2014-05-30 23:30'
time_end = '2014-07-31 23:30'
ds = ds.sel(time_counter=slice(time_start, time_end)).load()

# 2. Internal Tide Extraction

In [ ]:
array_time = ds.ssh_igw.time_counter.values
nt = array_time.size

In [ ]:
# PARAMETERS # 
wint = np.ones(3*nt)
gaspari = gaspari_cohn(np.arange(0,2*nt,1),nt,nt)
wint[:nt]=gaspari[:nt]
wint[2*nt:]=gaspari[nt:]

dt = 3600 # seconds

w = fp.fftfreq(3*nt,dt)# seconds^-1
nw = w.size

w1 = 1/15/3600
w2 = 1/9/3600
H = (np.abs(w)>w1) & (np.abs(w)<w2)
w_filtered = H*w

In [ ]:
idx_ocean = np.where(np.invert(np.isnan(ds.ssh_igw[0,:,:].values))) # indexes of ocean pixels 

In [ ]:
# PROCESSING # 
ssh_it_flat = np.array(Parallel(n_jobs=-1,backend='multiprocessing')(delayed(extract_it)(ds.ssh_igw[:,i,j],wint,H) for i,j in zip(idx_ocean[0],idx_ocean[1])))

In [ ]:
ssh_it_flat

In [ ]:
# ARRAY TO STORE THE RESULTS # 
array_ssh_it = np.nan*np.ones_like(ds.ssh_igw.values,dtype="float64")
array_ssh_it[:,idx_ocean[0],idx_ocean[1]]=ssh_it_flat.T

#np.save(file="/bettik/bellemva/MITgcm/ssh_it.npy",arr=array_ssh_it)

In [ ]:
del ssh_it_flat

In [ ]:
plt.figure()
plt.pcolormesh(ds.ssh_igw[1000,:,:])
plt.show()

In [ ]:
plt.figure()
plt.pcolormesh(array_ssh_it[1000,:,:])
plt.show()

In [ ]:
np.save(file="/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it.npy", arr=array_ssh_it)


In [ ]:
array_ssh_it=np.load("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it.npy")

In [ ]:
array_ssh_it

In [ ]:
# Create the date array
start_date = np.datetime64("2014-05-01 00:30")
end_date = np.datetime64("2014-07-31 23:30")
date_array = np.arange(start_date, end_date + np.timedelta64(1, 'h'), np.timedelta64(1, 'h'))

# Convert date array to pandas datetime index for easier monthly grouping
date_index = pd.to_datetime(date_array)

# Initialize variables for month tracking
current_month = date_index[0].month
month_start_idx = 0

# Loop through the date array
for i in range(1, len(date_index)):
    if date_index[i].month != current_month or i == len(date_index) - 1:
        # Define the end index of the current month
        month_end_idx = i if i != len(date_index) - 1 else i + 1

        # Create a dataset for the current month
        ds_it = xr.Dataset(
            data_vars={"ssh_it_tot": (["time_counter", "y", "x"], array_ssh_it[month_start_idx*24:month_end_idx*24, :, :])},
            coords={
                "nav_lat": (["y", "x"], ds.nav_lat.values),
                "nav_lon": (["y", "x"], ds.nav_lon.values),
                "time_counter": ds.time_counter.isel(time_counter=slice(month_start_idx*24, month_end_idx*24))
            }
        )

        # Save the dataset to a NetCDF file with month-based naming
        month_str = f"{current_month:02d}"  # Format month as two digits
        ds_it.to_netcdf(f"/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it_{month_str}.nc")
        print(f"Saved month: {month_str}")

        # Update the current month and start index for the next iteration
        current_month = date_index[i].month
        month_start_idx = i


In [ ]:
# Create the date array
start_date = np.datetime64("2014-05-01 00:30")
end_date = np.datetime64("2014-07-31 23:30")
date_array = np.arange(start_date, end_date + np.timedelta64(1, 'h'), np.timedelta64(1, 'h'))

# Convert date array to pandas datetime index for easier monthly grouping
date_index = pd.to_datetime(date_array)

# Initialize variables for month tracking
current_month = date_index[0].month
month_start_idx = 0

# Get unique months to iterate through
unique_months = pd.Series(date_index).dt.month.unique()

for month in unique_months:
    # Find the start and end indices for the current month
    month_mask = (pd.Series(date_index).dt.month == month)
    month_dates = date_index[month_mask]
    month_start_idx = np.where(date_index == month_dates[0])[0][0]
    month_end_idx = np.where(date_index == month_dates[-1])[0][0] + 1

    # Create a dataset for the current month
    ds_it = xr.Dataset(
        data_vars={"ssh_it": (["time_counter", "y", "x"], array_ssh_it[month_start_idx*24:month_end_idx*24, :, :])},
        coords={
            "nav_lat": (["y", "x"], ds.nav_lat.values),
            "nav_lon": (["y", "x"], ds.nav_lon.values),
            "time_counter": ds.time_counter.isel(time_counter=slice(month_start_idx*24, month_end_idx*24))
        }
    )

    # Save the dataset to a NetCDF file with month-based naming
    month_str = f"{month:02d}"  # Format month as two digits
    ds_it.to_netcdf(f"/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it_{month_str}.nc")
    print(f"Saved month: {month_str}")
